In [1]:
import json
from pathlib import Path
from typing import Literal

import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

In [2]:
URL = "https://www.paragraf.rs/propisi/zakon_o_radu.html"
output_path = Path("./srb_labor_law_data.json")
page = requests.get(URL)

In [3]:
soup = BeautifulSoup(page.content, "lxml")

In [4]:
labor_law = []
article_title = None
article_texts = []
article_link = None


def check_class_element(element, class_name: Literal["normal", "clan"]) -> bool:
    """Check if the element has a class 'normal'."""
    if class_name == "normal":
        return element.get("class") == ["normal"]
    else:
        return element.get("class") == ["clan"]


elements = soup.find_all("p")
for el in tqdm(elements):
    if check_class_element(el, "clan"):
        if article_title:
            labor_law.append(
                {"title": article_title, "texts": article_texts, "link": article_link}
            )
            article_texts = []
        article_title = el.get_text(strip=True)

        name_attr = el.find("a").get("name") if el.find("a") else None
        article_link = f"{URL}#{name_attr}" if name_attr else None
    elif article_title and check_class_element(el, "normal"):
        article_texts.append(el.get_text(strip=True))

if article_title and article_texts:
    labor_law.append(
        {"title": article_title, "texts": article_texts, "link": article_link}
    )

  0%|          | 0/1452 [00:00<?, ?it/s]

Write json

In [5]:
with open(output_path, "w", encoding="utf-8") as file:
    file.write(json.dumps(labor_law, indent=4))

Read json

In [17]:
with open(output_path, "r", encoding="utf-8") as file:
    data = json.loads(file.read())